Also we try to determine the optimal volume of the training sample.

- Threshold vas fixed at 0.024 and 0.00005
- We try to hold first kind error about 0.05
- We calculate first kind error and second kind error

**Here are results:**

| Training volume | Train time and UA | Err (thres 0,024) | Err (thres 0.00005) | Error (tune threshold) | Comments | Conclusion |
|--------|-------|--------|-------------------|---------------------------------------|------------|
| 877 616 | ~50h | - | - | - | First 30 log files  | Too long fitting (one thread) |
| 571 145 | ~30h | - | - | - | First 30 log files with filtered crawlers  | Too long fitting (one thread) |
| 151 839 | ~10h | - | - | - | First 10 log files with filtered crawlers | Not enough OS buffer space (all threads) |
| 20 000 | ~17m (~4K) | (0.143, 0.177) | (0.051, 0.226) | - | First 20K rows from first log file (without OneVsRest wrapper) | Same as without OneVsRest but slower:( but can use less memory |
| 20 000 | ~10m (~4K) | (0.143, 0.177) | (0.051, 0.226) | - | First 20K rows from first log file (use OneVsRest wrapper) | Pretty fine and fast train |
| 40 000 | ~30m (~6.5K) | (0.154, 0.158) | (0.051, 0.200) | - | First 40K from 10 filtered crawlers (use OneVsRest wrapper) | Better than 20K |
| 70 000 | ~60m (~9K) | (0.171, 0.136) | (0.057, 0.174) | 2e-5 (0.05, 0.19) | First 70K from 10 filtered crawlers (use OneVsRest wrapper) | Better than 40K |
| 100 000 | ~90m (~12K) | (0.178, 0.122) | (0.058, 0.161) | 1e-5 (0.05, 0.188) | First 100K from 10 filtered crawlers (use OneVsRest wrapper) | A little better than 70K |
| 130 000 | ~3h (~14.5K) | (0.167, 0.118) | (0.059, 0.157) | 9e-6 (0.05, 0.186) |First 130K from 10 filtered crawlers (use OneVsRest wrapper) | Almost equal 100K |

In [1]:
import pandas as pd
import numpy as np

import scipy.sparse
import sklearn.feature_extraction

import matplotlib.pylab as plt
%matplotlib inline

from tqdm import tqdm
import platform

pd.set_option("display.max_rows", 10)
pd.set_option('display.max_columns', 1100)

import os

%pylab inline
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


/home/levabd/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [124]:
main_data = np.load('df/main2_data.npy').tolist()[:20000]
values_data = np.load('df/values2_data.npy').tolist()[:20000]
order_data = np.load('df/order2_data.npy').tolist()[:20000]

main_df = pd.DataFrame(main_data)

main_df

,User_Agent,device_class_code,ip,os_code,os_family_code,timestamp,ua_class_code,ua_family_code,ua_version
0,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,desktop,62.89.6.193,windows_7,windows,1488578454,browser,chrome,chrome56.0.2924.87
1,Mozilla/5.0 (Windows NT 6.3; WOW64; rv:34.0) G...,desktop,62.210.80.74,windows_8_1,windows,1488578468,browser,firefox,firefox34.0
2,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,desktop,77.87.101.132,windows_7,windows,1488578472,browser,yandex-browser,yandex-browser17.1.1.1003
3,Mozilla/5.0 (Windows NT 5.1; rv:7.0.1) Gecko/2...,desktop,216.239.90.19,windows_xp,windows,1488578508,browser,firefox,firefox7.0.1
4,Mozilla/5.0 (Linux; Android 6.0.1; SAMSUNG SM-...,smartphone,37.19.72.34,android_6,android,1488578544,mobile_browser,mobile_samsung_browser,mobile_samsung_browser4.0
...,...,...,...,...,...,...,...,...,...
19995,Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537....,desktop,85.26.232.62,windows_10,windows,1488703450,browser,chrome,chrome56.0.2924.87
19996,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,desktop,46.118.53.27,windows_10,windows,1488703452,browser,yandex-browser,yandex-browser17.3.0.1785
19997,Mozilla/5.0 (Windows NT 6.1; rv:39.0) Gecko/20...,desktop,195.70.209.78,windows_7,windows,1488703453,browser,firefox,firefox39.0
19998,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,desktop,37.151.196.120,windows_7,windows,1488703458,browser,amigo,amigo54.0.2840.191


In [127]:
important_values_keys_set = {
    'Accept', 
    'Accept-Charset', 
    'Accept-Encoding'
}
important_orders_keys_set = {
    'Upgrade-Insecure-Requests',
    'Accept', 
    'If-Modified-Since',
    'Host', 
    'Connection', 
    'User-Agent', 
    'From', 
    'Accept-Encoding' 
}

orders_vectorizer = sklearn.feature_extraction.DictVectorizer(sparse=True, dtype=float)
values_vectorizer = sklearn.feature_extraction.DictVectorizer(sparse=True, dtype=float)

list_ua = pd.DataFrame(main_data).User_Agent.value_counts().index.tolist()

# For NaN Useragent
list_ua.append('NaN')

from lib.parsers.logParser import LogParser

l_parser = LogParser(log_folder='Logs/')
l_parser.reassign_orders_values(order_data, values_data)
full_sparce_dummy = l_parser.prepare_data(orders_vectorizer, values_vectorizer, important_orders_keys_set, important_values_keys_set, fit_dict=True)

from sklearn import preprocessing

lb = preprocessing.LabelBinarizer(sparse_output=True)
lb.fit(list_ua)
#y = lb.transform(pd.DataFrame(main_data).User_Agent.fillna('NaN'))
y = pd.DataFrame(main_data).User_Agent.fillna('NaN')
print("UA count: {}".format(len(list_ua)))

100%|██████████| 20000/20000 [00:00<00:00, 618419.12it/s]


Sparse dummy orders shape: 
(20000, 48)
Sparse dummy values shape: 
(20000, 108)
UA count: 4057


### Train model

In [128]:
%%time

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

#smart_clf = OneVsRestClassifier(LogisticRegression(C=100), n_jobs=-1)
smart_clf = LogisticRegression(C=100)
smart_clf.fit(full_sparce_dummy, y)

CPU times: user 17min 3s, sys: 240 ms, total: 17min 4s
Wall time: 17min 6s


## Test part

### Prepare data (50/50 bots and human mixed)

In [129]:
# -- HUMANS --

l_parser = LogParser(log_folder='Logs/')

main_data, values_data, order_data = l_parser.parse_train_sample(45, 46)

# -- BOTS --

l_parser = LogParser(log_folder='Logs/')

main_bot_data = np.load('df/main2_bot_data.npy').tolist()
values_bot_data = np.load('df/values2_bot_data.npy').tolist()
order_bot_data = np.load('df/order2_bot_data.npy').tolist()

#main_all = pd.DataFrame(main_data[:20000] + main_bot_data[:20000].to_dict('records'))

# Norming
main_bot_data2 = []
for bot in main_bot_data:
    main_bot_data2.append({ 'User_Agent': bot[0] })

main_bot_data = main_bot_data2
    
main_all = pd.DataFrame(main_data[:20000] + main_bot_data[:20000])
main_all['is_human'] = False
main_all.loc[0:19999,'is_human'] = True

values_all = values_data[:20000] + values_bot_data[:20000]
order_all = order_data[:20000] + order_bot_data[:20000]

list_all_ua = main_all.User_Agent.value_counts().index.tolist()

# For NaN Useragent
list_all_ua.append('NaN')

l_parser.reassign_orders_values(order_all, values_all)

test_sparce_dummy = l_parser.prepare_data(orders_vectorizer, values_vectorizer, important_orders_keys_set, important_values_keys_set, fit_dict=False)

lb_test = preprocessing.LabelBinarizer(sparse_output=True)
lb_test.fit(list_all_ua)
#y_test = lb_test.transform(main_all.User_Agent.fillna('NaN'))
y_test = main_all.User_Agent.fillna('NaN')

test_sparce_dummy.shape

100%|██████████| 37263/37263 [00:00<00:00, 612586.87it/s]


Sparse dummy orders shape: 
(37263, 48)
Sparse dummy values shape: 
(37263, 108)


(37263, 156)

### Test predictions proba (Threshold 0.024)

In [133]:
from lib.thresholdPredictions import ThresholdPredictions

#pred = ThresholdPredictions(user_agent_list=lb.classes_.tolist(), full_user_agent_list=lb_test.classes_.tolist(), clf=smart_clf)
#y_test_names, y_predicted, compare_answers, is_bot, answers_count = pred.bot_predict(test_sparce_dummy, y_test, 0.024072, sparce_y=True)
pred = ThresholdPredictions(user_agent_list=smart_clf.classes_.tolist(), full_user_agent_list=lb_test.classes_.tolist(), clf=smart_clf)
y_test_names, y_predicted, compare_answers, is_bot, answers_count = pred.bot_predict(test_sparce_dummy, y_test, 0.024072, sparce_y=False, single_labels=True)

37263it [00:03, 9577.05it/s]


In [134]:
compare_frame = pd.concat(
    [
        pd.DataFrame(y_test_names),
        y_predicted, 
        pd.DataFrame(compare_answers),  
        #pd.DataFrame(main_all[['is_human']]), 
        pd.DataFrame(is_bot), 
        pd.DataFrame(answers_count)
    ], keys=['test', 'predicted', 'correctness', 'is_bot_predicted', 'count'], axis=1, join='inner')

    
compare_frame['is_human'] = False
compare_frame.loc[0:19999,'is_human'] = True
compare_frame

,test,predicted,correctness,is_bot_predicted,count,is_human
,User_Agent,0,0,0,0,
0,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,[Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWeb...,False,True,8,True
1,Mozilla/5.0 (compatible; MSIE 9.0; Windows Pho...,[Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:...,False,False,5,True
2,Mozilla/5.0 (compatible; YandexBot/3.0; +http:...,[Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebK...,False,False,1,True
3,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,[Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWeb...,True,False,8,True
4,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:51.0) G...,[Mozilla/5.0 (Windows NT 10.0; WOW64; rv:51.0)...,True,False,5,True
...,...,...,...,...,...,...
37258,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,[Mozilla/5.0 (Windows NT 10.0; rv:48.0) Gecko/...,False,True,4,False
37259,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,[Mozilla/5.0 (Linux; Android 5.1.1; SM-J320F B...,False,False,10,False
37260,Mozilla/4.0 (compatible; MSIE 6.0; Windows NT ...,[Mozilla/5.0 (Windows NT 10.0; WOW64; rv:51.0)...,False,True,5,False


In [135]:
from sklearn.metrics import confusion_matrix, roc_curve, auc

headers_cm = confusion_matrix(~compare_frame.is_human, compare_frame.is_bot_predicted)

In [136]:
headers_FP = headers_cm.sum(axis=0) - np.diag(headers_cm)  
headers_FN = headers_cm.sum(axis=1) - np.diag(headers_cm)
headers_TP = np.diag(headers_cm)
headers_TN = headers_cm.sum() - (headers_FP + headers_FN + headers_TP)

In [137]:
print('TP: {}'.format(headers_TP))
print('TN: {}'.format(headers_TN))
print("FP: {}".format(headers_FP))
print("FN: {}".format(headers_FN))
print("Accuracy (ACC): {}".format((headers_TP + headers_TN) / (headers_TP + headers_TN + headers_FP + headers_FN)))
print("Sensitivity, hit rate, recall, or true positive rate (TPR): {}".format(headers_TP / (headers_TP + headers_FN)))
print("Precision or positive predictive value (PPV): {}".format(headers_TP / (headers_TP + headers_FP)))

TP: [14656 10656]
TN: [10656 14656]
FP: [6607 5344]
FN: [5344 6607]
Accuracy (ACC): [ 0.67927971  0.67927971]
Sensitivity, hit rate, recall, or true positive rate (TPR): [ 0.7328      0.61727394]
Precision or positive predictive value (PPV): [ 0.68927245  0.666     ]


In [138]:
first_kind_error = []
second_kind_error = []
for i, bot_prediction in enumerate(is_bot):
    cur_first_kind_error = False
    cur_second_kind_error = False
    if bot_prediction and i<20000:
        cur_first_kind_error = True
    if not bot_prediction and i>19999:
        cur_second_kind_error = True
    first_kind_error.append(cur_first_kind_error)
    second_kind_error.append(cur_second_kind_error)    

compare_frame['first_kind_error'] = first_kind_error
compare_frame['second_kind_error'] = second_kind_error
compare_frame

,test,predicted,correctness,is_bot_predicted,count,is_human,first_kind_error,second_kind_error
,User_Agent,0,0,0,0,,,
0,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,[Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWeb...,False,True,8,True,True,False
1,Mozilla/5.0 (compatible; MSIE 9.0; Windows Pho...,[Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:...,False,False,5,True,False,False
2,Mozilla/5.0 (compatible; YandexBot/3.0; +http:...,[Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebK...,False,False,1,True,False,False
3,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,[Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWeb...,True,False,8,True,False,False
4,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:51.0) G...,[Mozilla/5.0 (Windows NT 10.0; WOW64; rv:51.0)...,True,False,5,True,False,False
...,...,...,...,...,...,...,...,...
37258,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,[Mozilla/5.0 (Windows NT 10.0; rv:48.0) Gecko/...,False,True,4,False,False,False
37259,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,[Mozilla/5.0 (Linux; Android 5.1.1; SM-J320F B...,False,False,10,False,False,True
37260,Mozilla/4.0 (compatible; MSIE 6.0; Windows NT ...,[Mozilla/5.0 (Windows NT 10.0; WOW64; rv:51.0)...,False,True,5,False,False,False


In [139]:
print('Ошибка первого рода (когда мы принимаем нормального пользователя за бота): {}'.format(sum(first_kind_error) / y_test.shape[0]))
print('Ошибка второго рода (когда мы принимаем бота за нормального пользователя): {}'.format(sum(second_kind_error) / y_test.shape[0]))

Ошибка первого рода (когда мы принимаем нормального пользователя за бота): 0.1434130370608915
Ошибка второго рода (когда мы принимаем бота за нормального пользователя): 0.17730724847704157


### Threshold 0.00005

In [140]:
#pred = ThresholdPredictions(user_agent_list=lb.classes_.tolist(), full_user_agent_list=lb_test.classes_.tolist(), clf=smart_clf)
#y_test_names, y_predicted, compare_answers, is_bot, answers_count = pred.bot_predict(test_sparce_dummy, y_test, 0.00005, sparce_y=True)
pred = ThresholdPredictions(user_agent_list=smart_clf.classes_.tolist(), clf=smart_clf)
y_test_names, y_predicted, compare_answers, is_bot, answers_count = pred.bot_predict(test_sparce_dummy, y_test, 0.00005, sparce_y=False, single_labels=True)

37263it [00:04, 9065.76it/s]


In [141]:
compare_frame = pd.concat(
    [
        pd.DataFrame(y_test_names),
        y_predicted, 
        pd.DataFrame(compare_answers),  
        #pd.DataFrame(main_all[['is_human']]), 
        pd.DataFrame(is_bot), 
        pd.DataFrame(answers_count)
    ], keys=['test', 'predicted', 'correctness', 'is_bot_predicted', 'count'], axis=1, join='inner')

    
compare_frame['is_human'] = False
compare_frame.loc[0:19999,'is_human'] = True

headers_cm = confusion_matrix(~compare_frame.is_human, compare_frame.is_bot_predicted)

headers_FP = headers_cm.sum(axis=0) - np.diag(headers_cm)  
headers_FN = headers_cm.sum(axis=1) - np.diag(headers_cm)
headers_TP = np.diag(headers_cm)
headers_TN = headers_cm.sum() - (headers_FP + headers_FN + headers_TP)

print('TP: {}'.format(headers_TP))
print('TN: {}'.format(headers_TN))
print("FP: {}".format(headers_FP))
print("FN: {}".format(headers_FN))
print("Accuracy (ACC): {}".format((headers_TP + headers_TN) / (headers_TP + headers_TN + headers_FP + headers_FN)))
print("Sensitivity, hit rate, recall, or true positive rate (TPR): {}".format(headers_TP / (headers_TP + headers_FN)))
print("Precision or positive predictive value (PPV): {}".format(headers_TP / (headers_TP + headers_FP)))

first_kind_error = []
second_kind_error = []
for i, bot_prediction in enumerate(is_bot):
    cur_first_kind_error = False
    cur_second_kind_error = False
    if bot_prediction and i<20000:
        cur_first_kind_error = True
    if not bot_prediction and i>19999:
        cur_second_kind_error = True
    first_kind_error.append(cur_first_kind_error)
    second_kind_error.append(cur_second_kind_error)    

compare_frame['first_kind_error'] = first_kind_error
compare_frame['second_kind_error'] = second_kind_error

print('Ошибка первого рода (когда мы принимаем нормального пользователя за бота): {}'.format(sum(first_kind_error) / y_test.shape[0]))
print('Ошибка второго рода (когда мы принимаем бота за нормального пользователя): {}'.format(sum(second_kind_error) / y_test.shape[0]))

TP: [18194  8812]
TN: [ 8812 18194]
FP: [8451 1806]
FN: [1806 8451]
Accuracy (ACC): [ 0.72474036  0.72474036]
Sensitivity, hit rate, recall, or true positive rate (TPR): [ 0.9097      0.51045589]
Precision or positive predictive value (PPV): [ 0.6828298   0.82991147]
Ошибка первого рода (когда мы принимаем нормального пользователя за бота): 0.04846630706062314
Ошибка второго рода (когда мы принимаем бота за нормального пользователя): 0.22679333387005876


### Дообучение

_Идея:_ **Брать логи за последние 10 дней (нормально можно обучить) и замешивать в выборку юзерагенты из старых выборок которые входят в топ 200 старой выборки, но не входят в топ 200 новой.**

Для выборки по времени у класса LogParser реализована загрузка спец тренировочной выборки `parse_train_sample_by_time`